In [1]:
#!git clone https://github.com/citiususc/stac.git
#!mv stac stac_repo
#!mv stac_repo/stac ./

In [2]:
# Retrieve results and calculate the mean for each horizon

In [3]:
from tqdm import tqdm_notebook as tqdm
import os
import json

json_results = []
folder="../cal_new_training/"
for f in tqdm(os.listdir(folder)):
    json_results.append(json.load(open(folder+f)))
    
folder="../dt_2021_cal_results/json_files/"
for f in tqdm(os.listdir(folder)):
    json_results.append(json.load(open(folder+"/"+f)))

<ipython-input-3-c14d0e3c9a8b>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for f in tqdm(os.listdir(folder)):


<ipython-input-3-c14d0e3c9a8b>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for f in tqdm(os.listdir(folder)):


In [4]:
import pandas as pd

df = pd.DataFrame(json_results)
df.columns=["horizon","method",2,3]
df["horizon"]=df["horizon"].astype(int)
for i,feature in enumerate(["rmse","mae","bias","corr","crps"]):
    df[feature]=df[2].apply(lambda e:e[feature])
df.drop([2,3],axis=1,inplace=True)

In [5]:
df.head()

,horizon,method,rmse,mae,bias,corr,crps
0,53,QRF,16.507382,12.063445,2.933839,0.634474,8.869292
1,14,QRF,32.575191,23.577260,8.978260,0.479660,16.280645
2,9,QGB,14.562235,10.874951,0.156874,0.861771,7.790607
3,2,QGBL,9.748084,6.562479,-0.150876,0.879781,4.839738
4,21,QGBL,16.048977,11.609801,3.026849,0.728436,8.378299


In [6]:
samples_df = df.pivot_table(index="horizon",columns="method",values="rmse",aggfunc='mean').reset_index()
samples_df.head()

method,horizon,DT,MLP,NGBOOST,QGB,QGBL,QKNN,QKNNL,QLR,QRF,QRFL
0,1,8.825736,8.867571,7.912177,8.029100,8.084565,12.354182,8.221821,8.072366,13.232702,7.994909
1,2,10.481014,11.056401,10.522202,9.904721,10.127769,12.677295,10.567959,10.774682,13.814538,10.706099
2,3,10.796114,11.286370,10.658766,9.596696,9.752112,12.237881,10.588437,11.039470,13.656065,10.914558
3,4,10.983100,11.058686,11.028224,9.554046,9.753460,12.321590,11.089469,11.412586,13.985515,11.256652
4,5,11.424063,11.839607,11.196776,10.050412,10.009185,12.843222,11.520128,11.946266,14.591992,11.881897


In [15]:
order_methods="""QRF
QRFL
QKNN
QKNNL
QGB
QGBL
QLR
MLP
NGBOOST
DT""".split("\n")

In [17]:
import stac
samples = []
methods = df["method"].unique()
for method in methods:
    samples.append(samples_df[method])
ranks = stac.quade_test(*samples)
ranks_df = pd.DataFrame(zip(methods,ranks[3])).sort_values(1)
ranks_df.index=ranks_df[0]
ranks_df=ranks_df.loc[order_methods]

In [18]:
print(ranks_df.to_latex(index=None))

\begin{tabular}{lr}
\toprule
       0 &        1 \\
     QRF & 7.354393 \\
\midrule
    QRFL & 4.888386 \\
    QKNN & 6.522557 \\
   QKNNL & 3.379697 \\
     QGB & 0.967290 \\
    QGBL & 1.297199 \\
     QLR & 5.791688 \\
     MLP & 2.883587 \\
 NGBOOST & 3.442853 \\
      DT & 3.921512 \\
\bottomrule
\end{tabular}



In [8]:
ranks

(122.52940340210866,
 1.1102230246251565e-16,
 [10.0,
  1.3152542372881355,
  1.7638418079096045,
  4.68135593220339,
  7.875141242937853,
  8.868926553672317,
  3.92090395480226,
  4.595480225988701,
  6.646892655367232,
  5.332203389830508],
 [7.354392522921728,
  0.9672895928452985,
  1.2971985003707138,
  3.4428529064931888,
  5.791687987401467,
  6.522556713266966,
  2.8835866728291975,
  3.3796965413245954,
  4.8883857645296125,
  3.921511674086738])

In [ ]:
p-value is 1.1102230246251565e-16 . 